In [1]:
#导入基本库
import matplotlib.pyplot as plt
import random
import torch 
import torch.nn as nn
import numpy as np
import torch.utils.data as data
from torch.utils.data import DataLoader, TensorDataset, random_split
import tifffile as tiff #load tiff image
from PIL import Image
import os
import torch
from torchvision import transforms
from sklearn.preprocessing import MinMaxScaler

In [3]:
#放置图像数据数据导入模块
#建议将输入输出分别读取，读取结果为张量

In [ ]:
#假设目前已经分别获得了X,Y的张量
#我们要实现的功能是：随机选取50张图片进行特殊保存，方便后续可视化对比


#以下仅为代码示例：从1000张图片里分层抽出50张图片
#随机抽取50张图片出来做测试集
#由于张量是静态，需要先转换为numpy数组，再进行抽取和删减操作
all_picture=dataset_1.numpy() #print(all_picture.shape)

#抽取策略：分层抽样(比如：每20取1)
random_number = random.randint(0, 19)#随机生成一个0~19的随机数
#print(random_number)
index=[]
for i in range(50):
    index.append(20*i+random_number)
print(index)
#这里先提取索引对应图片数据
test_picture50=all_picture[index]
#print(test_picture50.shape)
#随后删除——将不再index中的索引图片copy到my_pictures中
my_pictures = np.delete(all_picture,index,axis=0)
print(my_pictures.shape)
#转换为tensor
dataset_1_1=torch.tensor(my_pictures)
print(dataset_1_1.shape)

In [ ]:
#接口说明，需要把参与训练的X集合与Y集合都转换为二维张量（例如，在第一个数据集上的shape格式应该为分别为【2979200，3】和【2979200，1】）（因为在上一步中已经提取出了50张图片作为后续可视化对比图）！！！
#注意要确保Y张量要为二维，否则要用语句规范，否则在后续会出现广播现象使其训练失败！！
# 示例
Y_batch = Y_batch.unsqueeze(1)  # 将目标张量调整为 (batch_size, 1)

In [ ]:
# 张量对数变换、归一化以及逆过程函数示例

#对数变换
# eg.对dataset_3进行对数变换，dataset_3.shape为【样本集数量，单个样本个数】
dataset_3_ln=torch.log(dataset_3+1)

#继续进行归一化操作
# 沿着适当的维度找到最小值和最大值,dim等于0（在竖直方向上）
min_values = torch.min(dataset_3_ln, dim=0).values
max_values = torch.max(dataset_3_ln, dim=0).values
print(min_values)
print(max_values)
# 对张量进行最小-最大归一化
normalized_dataset = (dataset_3_ln - min_values) / (max_values - min_values)
normalized_dataset=normalized_dataset.squeeze(0)
# 打印归一化后的张量
print(normalized_dataset.shape)

In [ ]:
#已经得到了得到完成标准化后的数据
#normalized_X：所有单个像素样本的输入集，是二维张量
#normalized_Y：所有单个像素样本的目标集，必须转换为二位张量的格式
#下面分割数据集

#首先缝合xy为数据集，只需替换normalized_X、normalized_Y为具体名字
dataset_all=TensorDataset(normalized_X, normalized_Y)
print(dataset_all)

#然后自己调整训练集、验证集、测试集比例
TRAIN_NUM=50000
VAL_NUM=10000
TEST_NUM=10000
#下面无需修改
train_dataset, val_dataset, test_dataset = data.random_split(
    dataset_all, [TRAIN_NUM, VAL_NUM,TEST_NUM]
    )
print(list(train_dataset))
print(list(val_dataset))
print(list(test_dataset))

In [ ]:
#加载dataloader，可以修改BATCH_SIZE（单次加载量）
#shuffle=True意为打乱数据，可以改False
BATCH_SIZE = 2000
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:

# 定义一个简单全连接网络
# 可以根据自己的实际情形修改
class FCN(nn.Module):
    def __init__(self):
        super(FCN, self).__init__()
        self.fc1 = nn.Linear(3, 1024)  # 第一个全连接层，输入大小为3，输出大小为1024
        self.fc2 = nn.Linear(1024, 1024)  # 第二个全连接层，输入大小为1024，输出大小为1024
        self.fc3 = nn.Linear(1024, 1)  # 第三个全连接层，输入大小为1024，输出大小为1（与展平后的Y大小匹配）

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # 使用ReLU激活函数的第一个全连接层
        x = torch.relu(self.fc2(x))  # 使用ReLU激活函数的第二个全连接层
        x = self.fc3(x)  # 第三个全连接层
        return x

#可以不更改
#使用MSEloss损失函数
# lr是学习率，可更改
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FCN().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
#正式训练
# Training loop
#设置epochs
NUM_EPOCHS = 15
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    for X_batch, Y_batch in train_loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, Y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation step
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for X_val, Y_val in val_loader:
            X_val, Y_val = X_val.to(device), Y_val.to(device)

            outputs = model(X_val)
            val_loss += criterion(outputs, Y_val).item()
        val_loss /= len(val_loader)

    print(f'Epoch {epoch+1}/{NUM_EPOCHS}, Train Loss: {total_loss/len(train_loader):.4f}, Validation Loss: {val_loss:.4f}')

print('Training completed!')

In [ ]:
# 测试集检验 
model.eval()
test_loss = 0
flag=0 #用于区分是否是第一次，

with torch.no_grad():
    for X_test, Y_test in test_loader:
        X_test, Y_test = X_test.to(device), Y_test.to(device)
        outputs = model(X_test)
        test_loss += criterion(outputs, Y_test).item()
        #下面模块是存储所有的测试集表现和目标值，分别到Outputs_all和Y_test_all中
        if flag==0:
            Outputs_all=outputs
            Y_test_all=Y_test
            flag+=1
        else:
            Outputs_all=torch.cat((Outputs_all,outputs),dim=0)
            Y_test_all=torch.cat((Y_test_all,Y_test),dim=0)
    test_loss /= len(test_loader)
print(f'Test Loss: {test_loss:.4f}')

#检查Y_test_all和Outputs_all的形状，便于后续画图
print(Y_test_all.shape)
print(Outputs_all.shape)

In [ ]:
#绘制测试集中的数据对比
error=Outputs_all-Y_test_all
eee=error.detach().numpy()
plt.hist(eee,30)
plt.hist

In [ ]:
# Assuming Outputs_all and Y_test_all are tensors
outputs_array = Outputs_all.detach().numpy()
Y_test_array = Y_test_all.detach().numpy()

# Create a scatter plot with a colormap
plt.scatter(outputs_array, Y_test_array, c=outputs_array, cmap='viridis', s=0.1)

# Add the y=x reference line
min_val = min(outputs_array.min(), Y_test_array.min())
max_val = max(outputs_array.max(), Y_test_array.max())
plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--', linewidth=2)

# Add color bar
plt.colorbar(label='Predicted Values')

plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.title('Scatter Plot of Predicted vs Actual Values')
plt.show()

In [ ]:
#下面是图像验证，利用一开始提取出的照片来进行图像验证
#思路：假设提取出用于可视化验证的的照片分为输入、输出保存为张量格式
#依旧是进行对数变换和归一化——别忘了避免广播




#屎山代码参考

#我的test_picture50.shape为(50, 4, 56, 56)
one_of_picture=test_picture50[3,:3,:,:]#把第一维改成1~50任意数字
print(one_of_picture.shape)#(3, 56, 56)
aim_of_truth=test_picture50[3,3,:,:]#把第一维改成1~50任意数字，与上面相同
print(aim_of_truth.shape)#(56, 56)
X_pic=torch.tensor(one_of_picture)
print(X_pic.shape)#torch.Size([3, 56, 56])
X_pic=X_pic.reshape(3,3136)
X_pic=X_pic.T
#对数变换
X_pic_ln=torch.log(X_pic+1)
print(min_values)
print(max_values)
# 对张量进行最小-最大归一化
X_pic_t = (X_pic_ln - min_values[:3]) / (max_values[:3] - min_values[:3])

# 打印归一化后的张量
print(X_pic_t.shape)
 

#用模型计算预测值
Y_outputs = model(X_pic_t)
print(Y_outputs.shape)#torch.Size([3136, 1])
#反归一化+反对数转换+转换为numpy格式以供画图
YY_ln=Y_outputs*(max_values[3]-min_values[3])+min_values[3]   #反归一化
YY=torch.exp(YY_ln)-1   #反对数转换
v=YY.reshape(56,56)
v_array=v.detach().numpy() #转换为numpy格式以供画图

In [ ]:
#对比图绘制示例

# 绘制灰度图



# 创建一个 1x2 的子图布局
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# 绘制第一个灰度图aim_of_truth(56*56)
axes[0].imshow(aim_of_truth, cmap='gray')
axes[0].set_title('Truth Image')
axes[0].axis('off')  # 隐藏坐标轴

# 绘制第二个灰度图v_array(56*56)
axes[1].imshow(v_array, cmap='gray')
axes[1].set_title('Predict Image')
axes[1].axis('off')  # 隐藏坐标轴

# 显示图像
plt.show()

In [ ]:
#绘制亮色对比图，可以改颜色eg.Reds
# 创建一个 1x2 的子图布局
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# 绘制第一个灰度图 aim_of_truth (56x56)
axes[0].imshow(aim_of_truth, cmap='Purples')
axes[0].set_title('Truth Image')
axes[0].axis('off')  # 隐藏坐标轴

# 绘制第二个灰度图 v_array (56x56)
axes[1].imshow(v_array, cmap='Purples')
axes[1].set_title('Predict Image')
axes[1].axis('off')  # 隐藏坐标轴

# 显示图像
plt.show()

In [ ]:
# 计算误差的语法格式
error = [i - j for i, j in zip(aim_of_truth, v_array)]
print(error)